In [80]:
!gdown --id 1vT1g4Crj6s6z2YNjSyPx5Fvz5KTpBYSB -O annottated_reddit.csv
!gdown --id 1N0U-i4PxT_DmTru0r7T5po-9f9cwGgB8 -O RedditComments2.csv
!gdown --id 1dlV7huXQfim8brL7LIBy-M2kFR63P2Oh -O Reddit_metadata2.csv

C:\Users\CACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gdown\cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1vT1g4Crj6s6z2YNjSyPx5Fvz5KTpBYSB
To: c:\Users\CACER\OneDrive\Desktop\deep-learning-final-project-yelp_reviews_classification\notebooks\annottated_reddit.csv

  0%|          | 0.00/1.51M [00:00<?, ?B/s]
 35%|███▍      | 524k/1.51M [00:00<00:00, 4.40MB/s]
100%|██████████| 1.51M/1.51M [00:00<00:00, 7.69MB/s]
C:\Users\CACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gdown\cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloadin

In [81]:
## Data processing and Data manipulation
import numpy as np # linear algenra
import pandas as pd # data processing

import sklearn 
from sklearn.model_selection import train_test_split

# Libraries and packages for NLP
import nltk
# It includes a set of text 
# processing libraries for classification, tokenization, 
# stemming, tagging, parsing, and semantic reasonin
import gensim
# library for unsupervised topic modeling, 
# document indexing, retrieval by similarity, and 
# other natural language processing functionalities, 
# using modern statistical machine learning.
from gensim.models import Word2Vec

import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import os
import sys

In [82]:
df = pd.read_csv("annottated_reddit.csv")
df.drop_duplicates(inplace= True)
df.dropna(inplace=True)

In [83]:
df.head()

,text,subreddit,bucket,annotator,annotation,confidence
0,MLP doesn't need to wait for a referendum to b...,europe,French politics,annotator00,Authority,Not Confident
1,Or - or - assclowns like Le Pen and Farage cou...,worldnews,French politics,annotator02,Equality,Confident
2,Congratulations on your victory Macron voters....,worldnews,French politics,annotator00,Care,Not Confident
3,The German Constitution did not let Hitler bec...,neoliberal,French politics,annotator02,Equality,Somewhat Confident
4,So Republicans really are for liberal policies...,politics,US Politics,annotator03,Equality,Confident


In [84]:
# Adding `src` directory to the directories for interpreter to search
sys.path.append(os.path.abspath(os.path.join('../..','w2v_utils.py')))


# Importing functions and classes from utility module
from w2v_utils import (Tokenizer,
                       w2v_trainer,
                       calculate_overall_similarity_score,
                       overall_semantic_sentiment_analysis
                       )


In [85]:
# Instancing the Tokenizer class
tokenizer = Tokenizer(clean= True,
                      lower= True, 
                      de_noise= True, 
                      remove_stop_words= True,
                      keep_negation=True)


In [86]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
df = df.drop(columns=['subreddit', 'bucket', 'annotator', 'confidence'])

In [89]:
df.head()

,text,annotation
0,MLP doesn't need to wait for a referendum to b...,Authority
1,Or - or - assclowns like Le Pen and Farage cou...,Equality
2,Congratulations on your victory Macron voters....,Care
3,The German Constitution did not let Hitler bec...,Equality
4,So Republicans really are for liberal policies...,Equality


In [113]:
df["annotation"].value_counts()

annotation
Care               973
Authority          814
Equality           618
Proportionality    472
Loyalty            306
Purity             231
Name: count, dtype: int64

In [112]:
# Dropping 
df = df[df["annotation"].str.contains("Equality,Proportionality") == False]
df = df[df["annotation"].str.contains("Purity,Equality") == False]
df = df[df["annotation"].str.contains("Purity,Proportionality") == False]
df = df[df["annotation"].str.contains("Loyalty,Equality") == False]
df = df[df["annotation"].str.contains("Purity,Loyalty") == False]
df = df[df["annotation"].str.contains("Proportionality,Loyalty") == False]
df = df[df["annotation"].str.contains("Equality,Authority") == False]
df = df[df["annotation"].str.contains("Proportionality,Authority") == False]
df = df[df["annotation"].str.contains("Loyalty,Care") == False]
df = df[df["annotation"].str.contains("Loyalty,Purity") == False]
df = df[df["annotation"].str.contains("Purity,Loyalty,Equality") == False]


In [114]:
def classify_values(df, input_column, output_column):
    df[output_column] = df[input_column].apply(lambda x: 0 if (x == 'Care') else 1 if (x == 'Equality') else 2 if (x == 'Loyalty') else 3 if (x == 'Authority') else 4 if (x == 'Purity') else 5)

In [115]:
classify_values(df, "annotation", "labeled_data")

In [116]:
df.head()

,text,annotation,labeled_data
0,MLP doesn't need to wait for a referendum to b...,Authority,3
1,Or - or - assclowns like Le Pen and Farage cou...,Equality,1
2,Congratulations on your victory Macron voters....,Care,0
3,The German Constitution did not let Hitler bec...,Equality,1
4,So Republicans really are for liberal policies...,Equality,1


In [117]:
# Tokenize reviews
df['tokenized_vectors'] = df['text'].apply(tokenizer.tokenize)

df['tokenized_vectors_len'] = df['tokenized_vectors'].apply(len)
df['tokenized_vectors_len'].apply(np.log).describe()

count    3414.000000
mean        2.799016
std         0.610728
min         1.098612
25%         2.302585
50%         2.772589
75%         3.258097
max         4.442651
Name: tokenized_vectors_len, dtype: float64

In [118]:
df.head(3)

,text,annotation,labeled_data,tokenized_vectors,tokenized_vectors_len
0,MLP doesn't need to wait for a referendum to b...,Authority,3,"[mlp, NOTneed, wait, referendum, break, europe...",53
1,Or - or - assclowns like Le Pen and Farage cou...,Equality,1,"[assclowns, like, le, pen, farage, could, demo...",22
2,Congratulations on your victory Macron voters....,Care,0,"[congratulations, victory, macron, voters, kno...",36


In [119]:
# Training a Word2Vec model
keyed_vectors, keyed_vocab = w2v_trainer(df['tokenized_vectors'])

In [120]:
# Find the most similar words to "care/harm" 
keyed_vectors.most_similar(positive=['care','harm'], negative=[], topn=15)

[('children', 0.9999149441719055),
 ('without', 0.9998925924301147),
 ('op', 0.9998899102210999),
 ('sorry', 0.9998860359191895),
 ('around', 0.9998852610588074),
 ('respect', 0.9998831748962402),
 ('nta', 0.9998764395713806),
 ('behavior', 0.9998753666877747),
 ('friend', 0.9998745322227478),
 ('live', 0.9998738765716553),
 ('man', 0.999873697757721),
 ('sex', 0.9998733401298523),
 ('parents', 0.999872088432312),
 ('others', 0.9998708963394165),
 ('wife', 0.9998690485954285)]

In [121]:
care_harm_concepts = ['care', 'benefit', 'amity','caring','compassion', 'empath', 'guard', 'peace', 'protect', 'safe', 'secure', 'shelter', 'shield', 'sympathy', 'abuse', 'annihilate', 'attack', 'brutal', 'cruelty', 'crush', 'damage', 'destroy', 'detriment', 'endanger', 'fight', 'harm', 'hurt', 'kill'] 
care_concepts = [concept for concept in care_harm_concepts if concept in keyed_vocab]


In [123]:
# Find the most similar words to "Fairness/cheating" 
keyed_vectors.most_similar(positive=['fair','cheating'], negative=[], topn=15)

[('made', 0.9999108910560608),
 ('everything', 0.9999091029167175),
 ('making', 0.9999021887779236),
 ('maybe', 0.9998984932899475),
 ('police', 0.9998982548713684),
 ('whole', 0.9998943209648132),
 ('lack', 0.9998937249183655),
 ('tax', 0.9998927712440491),
 ('back', 0.9998916983604431),
 ('usually', 0.9998915195465088),
 ('rather', 0.9998907446861267),
 ('wife', 0.9998902678489685),
 ('old', 0.9998878240585327),
 ('person', 0.9998875260353088),
 ('society', 0.9998866319656372)]

In [124]:
fair_cheat_concepts = ['fair', 'balance', 'constant','egalitarian','equable', 'equal', 'equity', 'fairminded', 'honest', 'fairly', 'impartial', 'justice', 'tolerant', 'bias', 'bigotry', 'discrimination', 'dishonest', 'exclusion', 'favoritism', 'inequitable', 'injustice', 'preference', 'prejudice', 'segregation', 'unequal', 'unfair', 'unjust'] 
fair_concepts = [concept for concept in fair_cheat_concepts if concept in keyed_vocab]


In [125]:
keyed_vectors.most_similar(positive=['loyalty','betrayal'], negative=[], topn=15)

[('imagine', 0.9983861446380615),
 ('violence', 0.998374879360199),
 ('whole', 0.9983651041984558),
 ('animals', 0.9983639121055603),
 ('lot', 0.9983518719673157),
 ('since', 0.998348593711853),
 ('usually', 0.9983477592468262),
 ('anything', 0.9983435273170471),
 ('feelings', 0.9983433485031128),
 ('use', 0.9983403086662292),
 ('part', 0.9983401894569397),
 ('back', 0.9983378052711487),
 ('yes', 0.9983372092247009),
 ('everything', 0.9983369708061218),
 ('year', 0.9983366131782532)]

In [127]:
loyal_betrayal_concepts = ['ally', 'cadre', 'clique','cohort','collective', 'communal', 'community', 'comrade', 'devote', 'familial', 'families', 'family', 'fellow', 'group', 'deceive', 'enemy', 'foregin', 'immigrant', 'imposter', 'individual', 'jilt', 'miscreant', 'renegade', 'sequester', 'spy', 'terrorist'] 
loyal_concepts = [concept for concept in loyal_betrayal_concepts if concept in keyed_vocab]

In [128]:
keyed_vectors.most_similar(positive=['authority','destruction'], negative=[], topn=15)

[('use', 0.9995715618133545),
 ('part', 0.9995705485343933),
 ('yes', 0.9995692372322083),
 ('completely', 0.9995673298835754),
 ('NOTeven', 0.9995670914649963),
 ('sure', 0.9995666742324829),
 ('violence', 0.9995647072792053),
 ('obviously', 0.9995626211166382),
 ('given', 0.9995619654655457),
 ('getting', 0.9995616674423218),
 ('wrong', 0.9995594024658203),
 ('cause', 0.99955815076828),
 ('hell', 0.999557614326477),
 ('find', 0.999557614326477),
 ('justify', 0.9995567798614502)]

In [130]:
auth_sub_concepts = ['abide', 'allegiance', 'authority','class','command', 'compliant', 'control', 'defer', 'father', 'hierarchy', 'duty', 'honor', 'law', 'leader', 'agitate', 'alienate', 'defector', 'defiant', 'defy', 'denounce', 'disobey', 'disrespect', 'dissent', 'dissident', 'illegal', 'insubordinate', 'insurgent', 'obstruct'] 
auth_concepts = [concept for concept in auth_sub_concepts if concept in keyed_vocab]

In [133]:
keyed_vectors.most_similar(positive=['purity','shame'], negative=[], topn=15)

[('put', 0.9997683763504028),
 ('getting', 0.9997579455375671),
 ('mean', 0.9997470378875732),
 ('new', 0.9997463226318359),
 ('hell', 0.9997445344924927),
 ('could', 0.999735951423645),
 ('yes', 0.9997355937957764),
 ('since', 0.9997342824935913),
 ('clearly', 0.9997342228889465),
 ('workers', 0.9997329115867615),
 ('law', 0.9997326135635376),
 ('use', 0.999731719493866),
 ('certain', 0.9997316002845764),
 ('must', 0.9997310042381287),
 ('comment', 0.9997291564941406)]

In [134]:
san_degrad_concepts = ['austerity', 'celibate', 'chaste','church','clean', 'decent', 'holy', 'immaculate', 'innocent', 'modest', 'pious', 'pristine', 'pure', 'sacred', 'adultery', 'blemish', 'contagious', 'debase', 'debauchery', 'defile', 'desecrate', 'dirt', 'disease', 'disgust', 'exploitation', 'filth', 'gross', 'impiety'] 
san_concepts = [concept for concept in san_degrad_concepts if concept in keyed_vocab]

In [135]:
keyed_vectors.most_similar(positive=['liberty','oppression'], negative=[], topn=15)

[('based', 0.9990540742874146),
 ('media', 0.9990530610084534),
 ('white', 0.9990503191947937),
 ('clearly', 0.9990448355674744),
 ('high', 0.9990434646606445),
 ('policy', 0.9990425109863281),
 ('laws', 0.9990400075912476),
 ('workers', 0.9990375638008118),
 ('also', 0.9990370273590088),
 ('best', 0.9990361928939819),
 ('country', 0.9990361332893372),
 ('public', 0.9990347027778625),
 ('completely', 0.9990332126617432),
 ('values', 0.9990312457084656),
 ('oh', 0.9990310072898865)]

In [136]:
lib_opp_concepts = ['blameless', 'canon', 'character','commendable','correct', 'decent', 'doctrine', 'ethics', 'exemplary', 'good', 'goodness', 'honest', 'legal', 'integrity', 'bad', 'evil', 'immoral', 'indecent', 'offend', 'offensive', 'transgress', 'wicked', 'wretched', 'wrong'] 
lib_concepts = [concept for concept in lib_opp_concepts if concept in keyed_vocab]

In [137]:
# Calculating Semantic Sentiment Scores by OSSA model
overall_df_scores = overall_semantic_sentiment_analysis (keyed_vectors = keyed_vectors,
                                                   care_target_tokens= care_concepts, 
                                                   fair_target_tokens= fair_concepts,
                                                   loyal_target_tokens= loyal_concepts,
                                                   auth_target_tokens= auth_concepts,
                                                   san_target_tokens= san_concepts,
                                                   lib_target_tokens= lib_concepts,
                                                   doc_tokens = df['tokenized_vectors'])

In [138]:
# To store semantic sentiment store computed by OSSA model in df
df['overall_care'] = overall_df_scores[0] 
df['overall_fair'] = overall_df_scores[1] 
df['overall_loyal'] = overall_df_scores[2]
df['overall_auth'] = overall_df_scores[3]
df['overall_san'] = overall_df_scores[4]
df['overall_lib'] = overall_df_scores[5]
df['overall_max_score'] = overall_df_scores[6]
df['moral_foundations'] = overall_df_scores[7]

In [131]:
df.head(20)

,Unnamed: 0,text,annotation,lemmatized,labeled_data,tokenized_vectors,overall_care,overall_fair,overall_loyal,overall_auth,overall_san,overall_lib,overall_max_score,moral_foundations
1,1,That particular part of the debate is especial...,Purity,particular debate especially funny macron expl...,4,"[particular, part, debate, especially, funny, ...",0.753502,0.840363,0.861728,0.790605,0.876608,0.860197,0.876608,2.0
8,8,TBH Marion Le Pen would be better. Closet fasc...,Equality,tbh marion le pen well closet fascist vs flamb...,1,"[tbh, marion, le, pen, would, better, closet, ...",0.463816,0.534338,0.638900,0.543265,0.623580,0.606955,0.638900,2.0
12,12,The Le Pen brand of conservatism and classical...,Authority,le pen brand conservatism classical right wing...,3,"[le, pen, brand, conservatism, classical, righ...",0.655129,0.798543,0.837738,0.767958,0.807040,0.754511,0.837738,4.0
21,21,"Hey, fuck you. Us leftists will never support ...","Loyalty,Equality",hey fuck leftist support le pen especially con...,5,"[hey, fuck, us, leftists, never, support, le, ...",0.719094,0.725482,0.828602,0.727889,0.819091,0.766099,0.828602,1.0
22,22,"Hey, fuck you. Us leftists will never support ...",Purity,hey fuck leftist support le pen especially con...,4,"[hey, fuck, us, leftists, never, support, le, ...",0.719094,0.725482,0.828602,0.727889,0.819091,0.766099,0.828602,2.0
23,23,"Hey, fuck you. Us leftists will never support ...",Equality,hey fuck leftist support le pen especially con...,1,"[hey, fuck, us, leftists, never, support, le, ...",0.719094,0.725482,0.828602,0.727889,0.819091,0.766099,0.828602,2.0
24,24,Clearly there were enough to affect the result...,Equality,clearly affect result election mid west thankf...,1,"[clearly, enough, affect, results, election, m...",0.751401,0.818181,0.867205,0.836998,0.857448,0.773556,0.867205,2.0
27,27,You are simplifying it. Islam is not the sole ...,Care,simplify islam sole reason terrorism western c...,0,"[simplifying, islam, NOTthe, sole, reason, beh...",0.761447,0.829347,0.903385,0.838433,0.879134,0.779269,0.903385,5.0
28,28,You are simplifying it. Islam is not the sole ...,"Care,Loyalty",simplify islam sole reason terrorism western c...,5,"[simplifying, islam, NOTthe, sole, reason, beh...",0.761447,0.829347,0.903385,0.838433,0.879134,0.779269,0.903385,3.0
29,29,You are simplifying it. Islam is not the sole ...,Care,simplify islam sole reason terrorism western c...,0,"[simplifying, islam, NOTthe, sole, reason, beh...",0.761447,0.829347,0.903385,0.838433,0.879134,0.779269,0.903385,3.0


In [140]:
import importlib
import w2v_utils
importlib.reload(w2v_utils)
from w2v_utils import(evaluate_model)

In [156]:
np.where(pd.isnull(df['moral_foundations']))

(array([2320, 2321, 2322, ..., 3411, 3412, 3413], dtype=int64),)

In [160]:
df = df[df['moral_foundations'].notna()]

In [161]:
# OSSA Model Evaluation
print("OSSA Model Evaluation: ")
evaluate_model(df['labeled_data'], 
               df['moral_foundations'])

print("=======================")

OSSA Model Evaluation: 
* Accuracy Score:  21.8966%
* F1 Score:  21.8966%
* Recall Score:  21.8966%
* Precision Score:  21.8966%


## Cleaned Data with Balance Data

In [6]:
## Data processing and Data manipulation
import numpy as np # linear algenra
import pandas as pd # data processing

import sklearn 
from sklearn.model_selection import train_test_split

# Libraries and packages for NLP
import nltk
# It includes a set of text 
# processing libraries for classification, tokenization, 
# stemming, tagging, parsing, and semantic reasonin
import gensim
# library for unsupervised topic modeling, 
# document indexing, retrieval by similarity, and 
# other natural language processing functionalities, 
# using modern statistical machine learning.
from gensim.models import Word2Vec

import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import os
import sys

In [228]:
df_new = pd.read_csv("C:\\Users\\CACER\\OneDrive\\Desktop\\lemmatized_reddit_og.csv")
df_new.drop_duplicates(inplace= True)
df_new.dropna(inplace=True)

In [229]:
df_new.head()

,Unnamed: 0,text,subreddit,bucket,annotator,annotation,confidence,lemmatized
0,0,That particular part of the debate is especial...,europe,French politics,annotator03,Non-Moral,Confident,particular debate especially funny macron expl...
1,1,That particular part of the debate is especial...,europe,French politics,annotator01,Purity,Confident,particular debate especially funny macron expl...
2,2,That particular part of the debate is especial...,europe,French politics,annotator02,Thin Morality,Confident,particular debate especially funny macron expl...
3,3,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator03,Non-Moral,Confident,france pretty lively lingo usually deliberatel...
4,4,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator00,Non-Moral,Somewhat Confident,france pretty lively lingo usually deliberatel...


In [230]:
df_new = df_new.drop(columns=['subreddit', 'bucket', 'annotator', 'confidence'])


In [232]:
df_new.head()

,Unnamed: 0,text,annotation,lemmatized
0,0,That particular part of the debate is especial...,Non-Moral,particular debate especially funny macron expl...
1,1,That particular part of the debate is especial...,Purity,particular debate especially funny macron expl...
2,2,That particular part of the debate is especial...,Thin Morality,particular debate especially funny macron expl...
3,3,"/r/france is pretty lively, with it's own ling...",Non-Moral,france pretty lively lingo usually deliberatel...
4,4,"/r/france is pretty lively, with it's own ling...",Non-Moral,france pretty lively lingo usually deliberatel...


In [301]:
df_new = df_new.drop(df_new[df_new['annotation'] == 'Authority'].sample(frac=0.11).index)


In [302]:
df_new["annotation"].value_counts()

Authority          863
Loyalty            857
Equality           854
Care               854
Purity             845
Proportionality    829
Name: annotation, dtype: int64

In [234]:
# Dropping 
df_new = df_new[df_new["annotation"].str.contains("Care,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Authority,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Care") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Purity,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Equality,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Purity,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Authority,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Loyalty,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Loyalty,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Care") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Care") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Equality,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Proportionality ") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Loyalty,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Loyalty,Purity,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Loyalty,Purity,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Purity,Authority,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Authority,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Loyalty,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Loyalty,Authority,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Loyalty,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care,Purity,Loyalty,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Equality,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Proportionality,Care,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Purity,Equality,Loyalty,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Proportionality,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Care,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Purity,Equality,Authority,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Care,Equality ") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Equality,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Equality,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Care,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care,Purity,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Loyalty,Care") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Loyalty,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Equality,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Proportionality,Care") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Authority,Equality,Proportionality") == False]
df_new = df_new[df_new["annotation"].str.contains("Loyalty,Equality,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Purity,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Proportionality,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Care") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Care,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Care,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Equality,Purity") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Proportionality,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Care,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Equality,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Loyalty,Authority,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care,Loyalty,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Loyalty,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Purity,Loyalty,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Non-Moral") == False]
df_new = df_new[df_new["annotation"].str.contains("Thin Morality") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Non-Moral") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care,Thin Morality,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Care,Thin Morality,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Proportionality,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Authority,Loyalty") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Purity,Loyalty,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Purity,Proportionality,Equality") == False]
df_new = df_new[df_new["annotation"].str.contains("Equality,Loyalty,Authority") == False]
df_new = df_new[df_new["annotation"].str.contains("Proportionality,Care,Loyalty,Authority") == False]

In [303]:
def classify_values(df, input_column, output_column):
    df[output_column] = df[input_column].apply(lambda x: 0 if (x == 'Care') else 1 if (x == 'Equality') else 2 if (x == 'Loyalty') else 3 if (x == 'Authority') else 4 if (x == 'Purity') else 5)

In [304]:
classify_values(df_new, "annotation", "labeled_data")

In [305]:
df_new.head()

,Unnamed: 0,text,annotation,lemmatized,labeled_data
1,1,That particular part of the debate is especial...,Purity,particular debate especially funny macron expl...,4
22,22,"Hey, fuck you. Us leftists will never support ...",Purity,hey fuck leftist support le pen especially con...,4
42,42,&gt; Valls is such a disgusting traitor to his...,Loyalty,valls disgusting traitor party help macron win...,2
44,44,&gt; Valls is such a disgusting traitor to his...,Loyalty,valls disgusting traitor party help macron win...,2
46,46,Melenchon's party said they'll hold an interna...,Loyalty,melenchon party say hold internal referendum d...,2


In [306]:
# Adding `src` directory to the directories for interpreter to search
sys.path.append(os.path.abspath(os.path.join('../..','w2v_utils.py')))


# Importing functions and classes from utility module
from w2v_utils import (Tokenizer,
                       w2v_trainer,
                       calculate_overall_similarity_score,
                       overall_semantic_sentiment_analysis, 
                       evaluate_model
                       )


In [307]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [308]:
# Instancing the Tokenizer class
tokenizer = Tokenizer(clean= True,
                      lower= True, 
                      de_noise= True, 
                      remove_stop_words= True,
                      keep_negation=True)

In [309]:
# Tokenize reviews
df_new['tokenized_vectors'] = df_new['lemmatized'].apply(tokenizer.tokenize)

In [311]:
# Training a Word2Vec model
keyed_vectors, keyed_vocab = w2v_trainer(df_new['tokenized_vectors'])

In [312]:
# Find the most similar words to "care/harm" 
keyed_vectors.most_similar(positive=['care','harm'], negative=[], topn=15)

[('set', 0.9997982382774353),
 ('test', 0.999783456325531),
 ('normal', 0.9997734427452087),
 ('ass', 0.9997714161872864),
 ('fire', 0.9997702836990356),
 ('lie', 0.9997661113739014),
 ('lack', 0.9997650384902954),
 ('employee', 0.9997637271881104),
 ('protect', 0.9997631907463074),
 ('responsibility', 0.999762237071991),
 ('son', 0.9997559785842896),
 ('possible', 0.9997543692588806),
 ('house', 0.9997540712356567),
 ('absolutely', 0.9997536540031433),
 ('healthy', 0.9997531771659851)]

In [313]:
care_harm_concepts = ['care', 'benefit', 'amity','caring','compassion', 'empath', 'guard', 'peace', 'protect', 'safe', 'secure', 'shelter', 'shield', 'sympathy', 'abuse', 'annihilate', 'attack', 'brutal', 'cruelty', 'crush', 'damage', 'destroy', 'detriment', 'endanger', 'fight', 'harm', 'hurt', 'kill'] 
care_concepts = [concept for concept in care_harm_concepts if concept in keyed_vocab]

In [314]:
# Find the most similar words to "Fairness/cheating" 
keyed_vectors.most_similar(positive=['fair','avoid'], negative=[], topn=15)

[('demand', 0.999862551689148),
 ('bit', 0.999858021736145),
 ('authority', 0.999854564666748),
 ('level', 0.9998540878295898),
 ('sound', 0.9998457431793213),
 ('ability', 0.9998427033424377),
 ('idea', 0.9998414516448975),
 ('lack', 0.999841034412384),
 ('action', 0.9998390078544617),
 ('accept', 0.9998366832733154),
 ('daily', 0.9998350143432617),
 ('kind', 0.9998345375061035),
 ('write', 0.9998315572738647),
 ('learn', 0.9998311996459961),
 ('possible', 0.9998305439949036)]

In [330]:
# To make sure that all 'fairness_cheating_concepts' are in the keyed word2vec vovabulary
# Here we are added more words that are associated into the positive words vector
fair_cheat_concepts = ['level','equally','fair', 'balance', 'constant','egalitarian','equable', 'equal', 'equity', 'fairminded', 'honest', 'fairly', 'impartial', 'justice', 'tolerant', 'bias', 'bigotry', 'discrimination', 'dishonest', 'exclusion', 'favoritism', 'inequitable', 'injustice', 'preference', 'prejudice', 'segregation', 'unequal', 'unfair', 'unjust'] 
fair_concepts = [concept for concept in fair_cheat_concepts if concept in keyed_vocab]

In [316]:
# Find the most similar words to "loyalty/betrayal" 
keyed_vectors.most_similar(positive=['loyalty','false'], negative=[], topn=15)

[('turn', 0.9998564124107361),
 ('history', 0.9998404383659363),
 ('r', 0.9998374581336975),
 ('major', 0.9998307228088379),
 ('remember', 0.9998297691345215),
 ('war', 0.9998262524604797),
 ('hold', 0.9998218417167664),
 ('blame', 0.9998170733451843),
 ('message', 0.999815821647644),
 ('ok', 0.999809980392456),
 ('course', 0.9998095035552979),
 ('citizen', 0.9998091459274292),
 ('bring', 0.9998080134391785),
 ('argue', 0.9998075366020203),
 ('share', 0.9998050332069397)]

In [317]:
# To make sure that all 'loyal_betrayal_concepts' are in the keyed word2vec vovabulary
# Here we are added more words that are associated into the positive words vector
loyal_betrayal_concepts = ['ally', 'cadre', 'clique','cohort','collective', 'communal', 'community', 'comrade', 'devote', 'familial', 'families', 'family', 'fellow', 'group', 'deceive', 'enemy', 'foregin', 'immigrant', 'imposter', 'individual', 'jilt', 'miscreant', 'renegade', 'sequester', 'spy', 'terrorist'] 
loyal_concepts = [concept for concept in loyal_betrayal_concepts if concept in keyed_vocab]

In [318]:
# Find the most similar words to "Authority/Subversion" 
keyed_vectors.most_similar(positive=['authority','destruction'], negative=[], topn=15)

[('test', 0.9998120665550232),
 ('normal', 0.9998102784156799),
 ('employee', 0.9998093843460083),
 ('play', 0.999809205532074),
 ('lack', 0.9998088479042053),
 ('kind', 0.9998056888580322),
 ('mother', 0.9998041987419128),
 ('house', 0.9998036026954651),
 ('fire', 0.9998021125793457),
 ('service', 0.9998015761375427),
 ('write', 0.9997997283935547),
 ('dog', 0.9997979402542114),
 ('huge', 0.999797523021698),
 ('personal', 0.9997972249984741),
 ('level', 0.9997959733009338)]

In [319]:
# To make sure that all 'authority_subversion_concepts' are in the keyed word2vec vovabulary
# Here we are added more words that are associated into the positive words vector
auth_sub_concepts = ['abide', 'allegiance', 'authority','class','command', 'compliant', 'control', 'defer', 'father', 'hierarchy', 'duty', 'honor', 'law', 'leader', 'agitate', 'alienate', 'defector', 'defiant', 'defy', 'denounce', 'disobey', 'disrespect', 'dissent', 'dissident', 'illegal', 'insubordinate', 'insurgent', 'obstruct'] 
auth_concepts = [concept for concept in auth_sub_concepts if concept in keyed_vocab]

In [320]:
# Find the most similar words to "sanctity/degradation" 
keyed_vectors.most_similar(positive=['faith','shame'], negative=[], topn=15)

[('bit', 0.9998696446418762),
 ('figure', 0.9998604655265808),
 ('demand', 0.9998602271080017),
 ('authority', 0.9998573660850525),
 ('learn', 0.9998548030853271),
 ('level', 0.9998518824577332),
 ('concern', 0.9998475313186646),
 ('action', 0.9998473525047302),
 ('lie', 0.9998448491096497),
 ('honestly', 0.9998417496681213),
 ('lack', 0.9998387694358826),
 ('idea', 0.9998369216918945),
 ('expect', 0.9998364448547363),
 ('fire', 0.9998348355293274),
 ('assume', 0.9998340606689453)]

In [321]:
# To make sure that all 'sancity_degrad_concepts' are in the keyed word2vec vovabulary
# Here we are added more words that are associated into the positive words vector
san_degrad_concepts = ['austerity', 'celibate', 'chaste','church','clean', 'decent', 'holy', 'immaculate', 'innocent', 'modest', 'pious', 'pristine', 'pure', 'sacred', 'adultery', 'blemish', 'contagious', 'debase', 'debauchery', 'defile', 'desecrate', 'dirt', 'disease', 'disgust', 'exploitation', 'filth', 'gross', 'impiety'] 
san_concepts = [concept for concept in san_degrad_concepts if concept in keyed_vocab]

In [322]:
# Find the most similar words to "liberty/oppression" 
keyed_vectors.most_similar(positive=['liberty','oppression'], negative=[], topn=15)

[('increase', 0.999697744846344),
 ('e', 0.9996851086616516),
 ('compare', 0.999667763710022),
 ('bunch', 0.9996671080589294),
 ('effect', 0.9996620416641235),
 ('oh', 0.9996575713157654),
 ('dangerous', 0.9996569752693176),
 ('include', 0.9996509552001953),
 ('charge', 0.9996486306190491),
 ('false', 0.9996448755264282),
 ('prove', 0.9996404051780701),
 ('bring', 0.9996398687362671),
 ('fight', 0.999635636806488),
 ('ban', 0.9996339678764343),
 ('stop', 0.9996328353881836)]

In [323]:
# To make sure that all 'liberty_oppression_concepts' are in the keyed word2vec vovabulary
# Here we are added more words that are associated into the positive words vector
lib_opp_concepts = ['blameless', 'canon', 'character','commendable','correct', 'decent', 'doctrine', 'ethics', 'exemplary', 'good', 'goodness', 'honest', 'legal', 'integrity', 'bad', 'evil', 'immoral', 'indecent', 'offend', 'offensive', 'transgress', 'wicked', 'wretched', 'wrong'] 
lib_concepts = [concept for concept in lib_opp_concepts if concept in keyed_vocab]

In [324]:
import importlib
import w2v_utils
importlib.reload(w2v_utils)
from w2v_utils import(calculate_overall_similarity_score,
                       overall_semantic_sentiment_analysis)

In [325]:
# Calculating Semantic Sentiment Scores by OSSA model
overall_df_scores = overall_semantic_sentiment_analysis (keyed_vectors = keyed_vectors,
                                                   care_target_tokens= care_concepts, 
                                                   fair_target_tokens= fair_concepts,
                                                   loyal_target_tokens= loyal_concepts,
                                                   auth_target_tokens= auth_concepts,
                                                   san_target_tokens= san_concepts,
                                                   lib_target_tokens= lib_concepts,
                                                   doc_tokens = df_new['tokenized_vectors'])

In [331]:
# To store semantic sentiment store computed by OSSA model in df
df_new['overall_care'] = overall_df_scores[0] 
df_new['overall_fair'] = overall_df_scores[1] 
df_new['overall_loyal'] = overall_df_scores[2]
df_new['overall_auth'] = overall_df_scores[3]
df_new['overall_san'] = overall_df_scores[4]
df_new['overall_lib'] = overall_df_scores[5]
df_new['overall_max_score'] = overall_df_scores[6]
df_new['moral_foundations'] = overall_df_scores[7]

In [332]:
df_new.head(10)

,Unnamed: 0,text,annotation,lemmatized,labeled_data,tokenized_vectors,overall_care,overall_fair,overall_loyal,overall_auth,overall_san,overall_lib,overall_max_score,moral_foundations
1,1,That particular part of the debate is especial...,Purity,particular debate especially funny macron expl...,4,"[particular, debate, especially, funny, macron...",0.997654,0.997649,0.998800,0.998503,0.998323,0.998392,0.998800,2
22,22,"Hey, fuck you. Us leftists will never support ...",Purity,hey fuck leftist support le pen especially con...,4,"[hey, fuck, leftist, support, le, pen, especia...",0.950842,0.950798,0.956711,0.955011,0.954246,0.954705,0.956711,2
42,42,&gt; Valls is such a disgusting traitor to his...,Loyalty,valls disgusting traitor party help macron win...,2,"[valls, disgusting, traitor, party, help, macr...",0.989156,0.989078,0.991785,0.991054,0.990676,0.990962,0.991785,2
44,44,&gt; Valls is such a disgusting traitor to his...,Loyalty,valls disgusting traitor party help macron win...,2,"[valls, disgusting, traitor, party, help, macr...",0.989156,0.989078,0.991785,0.991054,0.990676,0.990962,0.991785,2
46,46,Melenchon's party said they'll hold an interna...,Loyalty,melenchon party say hold internal referendum d...,2,"[melenchon, party, say, hold, internal, refere...",0.951920,0.951882,0.957774,0.956089,0.955262,0.955718,0.957774,2
47,47,Melenchon's party said they'll hold an interna...,Loyalty,melenchon party say hold internal referendum d...,2,"[melenchon, party, say, hold, internal, refere...",0.951920,0.951882,0.957774,0.956089,0.955262,0.955718,0.957774,2
50,50,I believe that Le Pen's desire to reduce Musli...,Authority,believe le pen desire reduce muslim immigratio...,3,"[believe, le, pen, desire, reduce, muslim, imm...",0.925600,0.925587,0.932880,0.930761,0.929776,0.930283,0.932880,2
54,54,Everything you're listing as problems with neo...,Authority,list problem neoliberalism consequence neolibe...,3,"[list, problem, neoliberalism, consequence, ne...",0.984132,0.984133,0.987451,0.986521,0.986036,0.986269,0.987451,2
76,76,I haven't been following the french election a...,Authority,follow french election sure reason vote candid...,3,"[follow, french, election, sure, reason, vote,...",0.967318,0.967280,0.972112,0.970740,0.970072,0.970461,0.972112,2
83,83,But it's not because of her euroscepticism tha...,Authority,euroscepticism vote le pen likely hate love po...,3,"[euroscepticism, vote, le, pen, likely, hate, ...",0.938337,0.938297,0.944930,0.943024,0.942144,0.942648,0.944930,2


In [333]:
df_new = df_new[df_new['moral_foundations'].notna()]

In [334]:
# OSSA Model Evaluation
print("OSSA Model Evaluation: ")
evaluate_model(df_new['labeled_data'], 
               df_new['moral_foundations'])

print("=======================")

OSSA Model Evaluation: 
* Accuracy Score:  17.3127%
* F1 Score:  17.3127%
* Recall Score:  17.3127%
* Precision Score:  17.3127%
